In [1]:
import csv, json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

from tqdm import tqdm

# 百度地图数据

In [2]:
read_csv_path = './source/data/CaseInfo_April/estate_info.csv'
out_geojson_path = './source/data/CaseInfo_April/estate_info.json'

In [13]:
df = pd.read_csv(read_csv_path)
df.head(3)

,address,lon,lat,name,city,district,town,encode
0,嘉定区万镇路1177弄,121.385610,31.258798,万镇社区嘉德坊,上海市,嘉定区,真新街道,210601001
1,嘉定区三里村,121.206128,31.459293,俆行镇三里村,上海市,嘉定区,徐行镇,210602001
2,嘉定区世盛路777号,121.191578,31.407024,上海五丰上食食品有限公司,上海市,嘉定区,徐行镇,210602002


In [4]:
latitude = 31.25
longitude = 121.385
latitude, longitude = map(float, (latitude, longitude))
latitude, longitude 

(31.25, 121.385)

In [19]:
features = []
for row in df.itertuples():
    address = getattr(row, "address")
    lon = getattr(row, "lon")
    lat = getattr(row, "lat")
    name = getattr(row, "name")
    city = getattr(row, "city")
    district = getattr(row, "district")
    town = getattr(row, "town")
    encode = getattr(row, "encode")
    
    features.append({
        "type": "Feature",
        "geometry":{
            "type":"Point",
            "coordinates":[lon, lat]
        },
        "properties":{
            'address': address,
            'name':name,
            'encode':encode,
            'city':city,
            'district':district,
            'town':town
            
        }
    })

save_data = {"type": "FeatureCollection", "features":features}

In [20]:
type(features)

list

In [21]:
import json
with open(out_geojson_path, "w", encoding='utf-8') as f:
    f.write(json.dumps(save_data, ensure_ascii=False, indent=4, separators=(',', ':') ) )

# 高德转WGS84

In [3]:
read_csv_path = './source/data/CaseInfo_April/estate_info_GD.csv'
out_geojson_path = './source/data/CaseInfo_April/estate_info_GD.json'

In [4]:
df = pd.read_csv(read_csv_path)
df.head(3)

,address,lon,lat,name,city,district,town,encode
0,嘉定区万镇路1177弄,121.378126,31.253413,真新新村嘉德坊,上海市,嘉定区,真新街道,210601001
1,嘉定区三里村,121.199999,31.452898,嘉定工业区三里村,上海市,嘉定区,徐行镇,210602001
2,嘉定区世盛路777号,121.185014,31.400883,上海五丰上食食品有限公司,上海市,嘉定区,徐行镇,210602002


In [5]:
from source.data.gcj02towgs84 import *

In [6]:
lon = 121.378126
lat = 31.253413

gcj02towgs84(lon, lat)

[121.37348669654547, 31.255250041309957]

In [7]:
features = []
for row in df.itertuples():
    address = getattr(row, "address")
    temp_lon = getattr(row, "lon")
    temp_lat = getattr(row, "lat")
    name = getattr(row, "name")
    city = getattr(row, "city")
    district = getattr(row, "district")
    town = getattr(row, "town")
    encode = getattr(row, "encode")
    
    lon, lat = gcj02towgs84(temp_lon, temp_lat) # TODO GCJ2WSG
    
    features.append({
        "type": "Feature",
        "geometry":{
            "type":"Point",
            "coordinates":[lon, lat]
        },
        "properties":{
            'address': address,
            'name':name,
            'encode':encode,
            'city':city,
            'district':district,
            'town':town
            
        }
    })

save_data = {"type": "FeatureCollection", "features":features}

In [8]:
import json
with open(out_geojson_path, "w", encoding='utf-8') as f:
    f.write(json.dumps(save_data, ensure_ascii=False, indent=4, separators=(',', ':') ) )